In [1]:
import pandas as pd
import numpy as np

from scipy.sparse import csr_matrix, save_npz

In [2]:
anime = pd.read_csv("data/anime_clean_noml.csv")

In [17]:
vectorLoad = np.load('data/kernelSimilarity.npz')
kernelSimilarity = vectorLoad[vectorLoad.files[0]]

In [21]:
def animeSearch(df_, nameQuery, n=5, sortByScore=True):
    df = df_.copy()
    nameQuery = nameQuery.lower()
    df["name_lower"] = df["Name"].apply(lambda x: x.lower())
    
    pd.set_option('display.max_rows', n)
    if n in ['all', 'All']:
        pd.set_option('display.max_rows', len(nameContains))
        
    nameContains = df.loc[df.name_lower.str.contains(nameQuery, na=False)].drop(columns=['Features',
                                                                                         'name_lower'])
    if sortByScore:
        nameContains = nameContains.sort_values(by="Score", ascending=False)
    
    return nameContains


def getHighestScoreAnime(df_, n=5, query=None):
    df = df_.copy()
    df["name_lower"] = df["Name"].apply(lambda x: x.lower())
    
    pd.set_option('display.max_rows', n)
    
    getHighest = df[df.Score != 'Unknown'].copy()
    getHighest["Score"] = getHighest["Score"].astype("float64")
    
    if query != None:
        getHighest = getHighest.sort_values(
            by="Score", ascending=False).query(query)[:n]
        print(
            f"Generated {getHighest.shape[0]} Rows, and {getHighest.shape[1]} Columns")
        return getHighest.drop(columns=['Features','name_lower'])
    
    getHighest = getHighest.sort_values(by="Score", ascending=False)[:n]
    
    print(
        f"Generated {getHighest.shape[0]} Rows, and {getHighest.shape[1]} Columns")
    
    return getHighest.drop(columns=['Features','name_lower'])


def filterByScore(df_, query=None, showAll=False, sort=True, ascending=False):
    df = df_.copy()
    df["name_lower"] = df["Name"].apply(lambda x: x.lower())
    
    getDf = df[df.Score != 'Unknown']
    getDf["Score"] = getDf["Score"].astype("float64")
    getDf = getDf.query(query)
    
    if sort:
        getDf = getDf.sort_values(by="Score", ascending=ascending)
    
    if showAll:
        pd.set_option('display.max_rows', len(getDf))
        print(f"Generated {getDf.shape[0]} Rows, and {getDf.shape[1]} Columns")
        return getDf.drop(columns=['Features','name_lower'])
    
    print(f"Generated {getDf.shape[0]} Rows, and {getDf.shape[1]} Columns")
    return getDf.drop(columns=['Features','name_lower'])


def filterByGenre(df_, genreContains, query=None, sortByScore=False, ascending=False, showAll=False):
    
    getGenres = df_.copy()
    getGenres["name_lower"] = getGenres["Name"].apply(lambda x: x.lower())
    getGenres = getGenres.loc[getGenres.Genres.str.contains(
        genreContains, na=False)]
    
    if query:
        getGenres = getGenres[getGenres.Score != 'Unknown']
        getGenres["Score"] = getGenres["Score"].astype("float64")
        getGenres = getGenres.query(query)
    
    if sortByScore:
        getGenres = getGenres[getGenres.Score != 'Unknown']
        getGenres["Score"] = getGenres["Score"].astype("float64")
        getGenres = getGenres.sort_values(by="Score", ascending=ascending)
    
    if showAll:
        pd.set_option('display.max_rows', len(getGenres))
        print(f"Filterting {genreContains} ...")
        print(
            f"Generated {getGenres.shape[0]} Rows, and {getGenres.shape[1]} Columns")
    
        return getGenres.drop(columns=["Features"])
    
    print(f"Filterting {genreContains} ...")
    print(
        f"Generated {getGenres.shape[0]} Rows, and {getGenres.shape[1]} Columns")
    
    return getGenres.drop(columns=['Features','name_lower'])


def getSimilarAnimeByIndex(index, df_=anime, model=kernelSimilarity, n=10, showAll=True):
    df = df_.copy()
    df["name_lower"] = df["Name"].apply(lambda x: x.lower())
    
    getSimilar = pd.DataFrame(model[index], columns=["Similarity"])
    filteringAboveZeros = getSimilar.query("Similarity > 0")
    sortSimilar = filteringAboveZeros.sort_values(
        by="Similarity", ascending=False)
    similarityScore = sortSimilar.iloc[1:n+1]
    similarAnime = similarityScore.index

    if showAll:
        pd.set_option('display.max_rows', n)
        return df.iloc[similarAnime].join(similarityScore).set_index('MAL_ID').sort_values(by="Score", ascending=False).drop(columns=['Features','name_lower'])

    return df.iloc[similarAnime].join(similarityScore).set_index('MAL_ID').sort_values(by="Score", ascending=False).drop(columns=['Features','name_lower'])


def getSimilarAnimeByName(name, df_=anime, model=kernelSimilarity, n=10, showAll=True):
    df = df_.copy()
    name = name.lower()
    df["name_lower"] = df["Name"].apply(lambda x: x.lower())
    
    getAnime = df.loc[df.name_lower.str.contains(name, na=False)].drop(
        columns=['Features']).sort_values(by="Score", ascending=False)
    getIndex = getAnime.index[0]
    getSimilar = pd.DataFrame(model[getIndex], columns=["Similarity"])
    filteringAboveZeros = getSimilar.query("Similarity > 0")
    sortSimilar = filteringAboveZeros.sort_values(
        by="Similarity", ascending=False)
    similarityScore = sortSimilar.iloc[1:n+1]
    similarAnime = similarityScore.index

    if showAll:
        pd.set_option('display.max_rows', n)
        return df.iloc[similarAnime].join(similarityScore).set_index('MAL_ID').sort_values(by="Score", ascending=False).drop(columns=['Features','name_lower'])

    return df.iloc[similarAnime].join(similarityScore).set_index('MAL_ID').sort_values(by="Score", ascending=False).drop(columns=['Features','name_lower'])


def getAnimeType(animeType, df_=anime):
    df = df_.copy()
    df["name_lower"] = df["Name"].apply(lambda x: x.lower())
    return df[df.Type == animeType].sort_values(by="Score", ascending=False).drop(columns=['Features','name_lower'])

In [19]:
animeResults = animeSearch(anime, nameQuery="Shingeki", n=5)
animeResults

,MAL_ID,Name,Score,Genres,Synopsis,Type,Popularity,Members,Favorites,Ranked,Episodes,Rating
14644,40028,Shingeki no Kyojin: The Final Season,9.17,"Action, Military, Mystery, Super Power, Drama,...",Gabi Braun and Falco Grice have been training ...,TV,119,733260,44862,2.0,16,R - 17+ (violence & profanity)
13717,38524,Shingeki no Kyojin Season 3 Part 2,9.10,"Action, Drama, Fantasy, Military, Mystery, Sho...",Seeking to restore humanity's diminishing hope...,TV,63,1073626,40985,4.0,10,R - 17+ (violence & profanity)
...,...,...,...,...,...,...,...,...,...,...,...,...
11681,35122,Shingeki no Kyotou,6.35,"Action, Fantasy, Shounen",special anime featuring a battle against the C...,Special,3833,18073,127,6630.0,1,PG-13 - Teens 13 or older
8755,28447,Shingeki no Bahamut: Genesis - Roundup,6.33,"Action, Adventure, Demons, Supernatural, Magic...",cap of the first six episodes of Shingeki no B...,Special,4354,13467,7,6727.0,1,R - 17+ (violence & profanity)


In [22]:
getSimilarAnimeByIndex(index=6550, n=5)

,Name,Score,Genres,Synopsis,Type,Popularity,Members,Favorites,Ranked,Episodes,Rating,Similarity
MAL_ID,,,,,,,,,,,,
40028,Shingeki no Kyojin: The Final Season,9.17,"Action, Military, Mystery, Super Power, Drama,...",Gabi Braun and Falco Grice have been training ...,TV,119,733260,44862,2.0,16,R - 17+ (violence & profanity),0.394277
38524,Shingeki no Kyojin Season 3 Part 2,9.10,"Action, Drama, Fantasy, Military, Mystery, Sho...",Seeking to restore humanity's diminishing hope...,TV,63,1073626,40985,4.0,10,R - 17+ (violence & profanity),0.314509
35760,Shingeki no Kyojin Season 3,8.59,"Action, Military, Mystery, Super Power, Drama,...","Still threatened by the ""Titans"" that rob them...",TV,48,1212430,14971,76.0,12,R - 17+ (violence & profanity),0.404391
25777,Shingeki no Kyojin Season 2,8.45,"Action, Military, Mystery, Super Power, Drama,...","For centuries, humanity has been hunted by gia...",TV,16,1591506,18262,128.0,12,R - 17+ (violence & profanity),0.393233
42091,Shingeki no Kyojin: Chronicle,7.68,"Action, Military, Mystery, Super Power, Drama,...",The compilation film will recap the anime's 59...,Movie,2249,50634,211,1089.0,1,R - 17+ (violence & profanity),0.351591


In [14]:
getSimilarAnimeByName("chuunibyou", n=20)

,Name,Score,Genres,Synopsis,Type,Popularity,Members,Favorites,Ranked,Similarity
MAL_ID,,,,,,,,,,
37450,Seishun Buta Yarou wa Bunny Girl Senpai no Yum...,8.38,"Comedy, Supernatural, Drama, Romance, School",The rare and inexplicable Puberty Syndrome is ...,TV,78,940033,29642,166.0,0.091608
14741,Chuunibyou demo Koi ga Shitai!,7.77,"Slice of Life, Comedy, Drama, Romance, School",Everybody has had that stage in their life whe...,TV,77,955524,17973,878.0,0.330204
18671,Chuunibyou demo Koi ga Shitai! Ren,7.56,"Comedy, Drama, Romance, School, Slice of Life","The awkward lovebirds, Yuuta Togashi and Rikka...",TV,226,522620,2952,1378.0,0.507584
16934,Chuunibyou demo Koi ga Shitai!: Kirameki no......,7.51,"Comedy, Drama, Romance, School, Slice of Life",lthough Yuuta Togashi and Rikka Takanashi have...,Special,901,164542,144,1551.0,0.473849
27601,Chuunibyou demo Koi ga Shitai! Ren: The Rikka ...,7.47,"Comedy, Drama, Romance, School, Slice of Life","One normal school day, Rikka Takanashi notices...",Special,1519,91158,88,1672.0,0.358478
19021,Takanashi Rikka Kai: Chuunibyou demo Koi ga Sh...,7.35,"Comedy, Drama, Romance, School, Slice of Life","Summary of the first season of the show, as se...",Movie,1739,75194,102,2109.0,0.647468
15687,Chuunibyou demo Koi ga Shitai! Lite,7.26,"Comedy, School",Short episodes aired on KyoAni's official YouT...,ONA,1306,109113,176,2520.0,0.416979
35847,SSSS.Gridman,7.19,"Action, Sci-Fi, Mecha",Yuuta Hibiki wakes up in the room of Rikka Tak...,TV,734,199953,937,2804.0,0.121330
22859,Takanashi Rikka Kai: Chuunibyou demo Koi ga Sh...,7.18,Comedy,Special episode included with the Blu-ray/DVD ...,Special,3112,27746,7,2840.0,0.639991
